# S3E25 Moh's Hardness Prediction EDA
* Predict Moh's hardness given physical parameters
* Regression Problem, try
    * Linear Regression Baseline
    * Lasso, Ridge
    * Support Vectors
    * Decision Trees
    * Random Forest
    * Gradient Boosting
    * ANN's (fastai tabular learner)


# Installs

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')


# Configuration

In [ ]:
class Config:
    seed = 12
    folds = 5

In [ ]:
config = Config()

# Load Data

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s3e25/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')
ss = pd.read_csv('/kaggle/input/playground-series-s3e25/sample_submission.csv')

# Top Level Statistics

In [ ]:
# print all sets - assuming train,test, ss
print(f'Training Data Shape: {train.shape}') 
display(train.head()) 
print(f'Test Data Shape: {test.shape}') 
display(test.head()) 
print(f'Sample Submission Shape: {ss.shape}') 
display(ss.head())

In [ ]:
display(train.describe())
display(test.describe())

Todo: Look at percentage difference between train and test

In [ ]:
display(train.info())
display(test.info())

### Takeaways
* Not a super huge dataset, 10K train, 7K test
* 10 features aside from id and 'hardness' in train as the target
* All floats except id
* No missing data

# EDA

## Distribution of Features

In [ ]:
features = [col for col in test.columns if test[col].dtype=='float']
rows = int(np.ceil(len(features)/3)) 
plt.figure(figsize=(16,5*rows),tight_layout=True) 
for i,col in enumerate(features): 
  plt.subplot(rows,3,i+1) 
  sns.histplot(train[col],kde=True) 
  sns.histplot(test[col], kde=True)
  plt.title(col)

### Takeaways
* a few features have large outliers, need to investigate to see if should remove
* Some quite skewed, try log or sqrt transform
* Train and test look pretty similar, should be decent agreement between cv and lb

## Distribution of Target

In [ ]:
sns.histplot(train.Hardness,kde=True)
plt.title('Distribution of Target Variable')
plt.show()

## Correlation

In [ ]:
sns.set(font_scale=.65)
plt.figure(figsize=(10, 5))
mask = np.triu(np.ones_like(train.corr(), dtype=bool)) 
heatmap = sns.heatmap(train.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG') 
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

### Takeaways
* There are some strongly correlated features
    * Try PCA

## Correlation with Target

In [ ]:
plt.figure(figsize=(4, 5))  # set size by num of features
heatmap = sns.heatmap(train.corr()[['Hardness']].sort_values(by='Hardness', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG') 
heatmap.set_title('Features Correlating with Target', fontdict={'fontsize':18}, pad=16);

### Takeways
* Seems like good signal
* look into dropping the 3 lower correlated ones

## Scatterplots with Target

In [ ]:
sns.set(font_scale=1.0)
features = [col for col in test.columns if test[col].dtype=='float']
rows = int(np.ceil(len(features)/3)) 
plt.figure(figsize=(16,5*rows),tight_layout=True) 

for i,col in enumerate(features): 
    plt.subplot(rows,3,i+1) 
    sns.scatterplot(x=train[col],y=train.Hardness)
    plt.title(col)

### Takeaways
* how to deal with the zeros?
    * run experiment to determine if removing them helps

# Preprocesing

In [ ]:
y = train.Hardness

# Cross Validation Functions

In [ ]:
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import KFold

def run_cv_and_predict(train, test, features, model, seed, verbose=True):
    
    folds = config.folds
    # initialize arrays 
    fold_scores = []
    oof = np.zeros(train.shape[0])
    preds = np.zeros((test.shape[0],folds))

    # setup folding strategy
    skf = KFold(n_splits=folds,random_state = seed,shuffle=True)

    # start cross validation
    cur_fold = 1
    for trn_idx, val_idx in skf.split(train[features], y):

        # split indicies into train and validation
        x_train = train[features].iloc[trn_idx]
        y_train = y.iloc[trn_idx]
        x_valid = train[features].iloc[val_idx]
        y_valid = y.iloc[val_idx]

        # fit model
        model.fit(x_train,y_train)

        # predict on validation set
        fold_preds = model.predict(x_valid)
        #fold_preds = fold_preds.reshape(len(fold_preds)) # sbabwtdt
        oof[val_idx] = fold_preds

        # Compute scores
        fold_score = median_absolute_error(y_valid,fold_preds)  # CHOOSE METRIC HERE
        fold_scores.append(fold_score)
        if verbose:
            print(f'MAE Score, fold {cur_fold}: {fold_score}')

        # predict on test set - store all fold preds (take mode later)
        test_preds = model.predict(test[features])
        #test_preds = test_preds.reshape(len(test_preds))  # shouldn't have to do this.
        preds[:,cur_fold-1] = test_preds  
        cur_fold +=1
    
    # Print mean fold and oof score 
    oof_score = median_absolute_error(y,oof)
    # oof_score = np.sqrt(mean_squared_error(y,oof))
    print(f'MAE score: {np.mean(fold_scores):.5f}, Stdev: {np.std(fold_scores):.5f}, OOF score: {oof_score:.5f}')
    # print(f'RMSE score: {np.mean(scores):.5f}, Stdev: {np.std(scores):.5f}, OOF score: {oof_score:.5f}')

    return (preds,fold_scores,oof_score,oof)

# Models

## Linear Regression Baseline

In [ ]:
from sklearn.linear_model import LinearRegression

# model_lr = LinearRegression()
# preds_lr,scores_lr,oof_score_lr,oof_lr = run_cv_and_predict(train,test,features,model_lr,config.seed)

# default cv = .96700, lb = .98495

## Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

# model_lasso = Lasso()
# preds_lasso,scores_lasso,oof_score_lasso,oof_lasso = run_cv_and_predict(train,test,features,model_lasso,config.seed)

# default: MAE score: 1.08483, Stdev: 0.01510, OOF score: 1.08529
# how is this above 1?

## Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

# model_svm = Ridge()
# preds_ridge,scores_ridge,oof_score_ridge,oof_ridge = run_cv_and_predict(train,test,features,model_ridge,config.seed)

#default: MAE score: 0.96897, Stdev: 0.01115, OOF score: 0.96980

## Support Vector Machines

In [ ]:
# use SVR for regression, SVC for classfication
# from sklearn.svm import SVR #SVC, LinearSVC

# model_svm = SVR(kernel='linear')
# preds_svm,scores_svm,oof_score_svm,oof_svm = run_cv_and_predict(train,test,features,model_svm,config.seed)

# svm default with rbf kernel: MAE score: 0.88030, Stdev: 0.01481, OOF score: 0.88354, lb = crashed
# svm with linear kernel: MAE score: 0.82035, Stdev: 0.01722, OOF score: 0.82318, lb = 

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# values = []
# # optimize max depth
# for md in range(3,30):

model_dt = DecisionTreeRegressor(max_depth=15, random_state=config.seed)
preds_dt,scores_dt,oof_score_dt,oof_dt = run_cv_and_predict(train,test,features,model_dt,config.seed,verbose=False)
#     values.append([md, np.mean(scores_dt)])
    
    
#max_depth 4: MAE score: 0.76808, Stdev: 0.05571, OOF score: 0.76048, 
# max depth 5: MAE score: 0.76159, Stdev: 0.02841, OOF score: 0.76522
# max depth 6: MAE score: 0.70609, Stdev: 0.03774, OOF score: 0.69927
# max depth 7: MAE score: 0.69753, Stdev: 0.00514, OOF score: 0.69643
# max depth 8: MAE score: 0.67461, Stdev: 0.02385, OOF score: 0.66893
# max depth 18: MAE score: 0.64324, Stdev: 0.07321, OOF score: 0.64942
# 28 is too high, optimizing 

# max depth 15 is min, MAE score: 0.63260, Stdev: 0.05069, OOF score: 0.62941, lb = .63083


In [ ]:
# md_df = pd.DataFrame(values,columns=['max_depth','cv_score'])
# plt.plot(md_df.max_depth,md_df.cv_score)
# plt.title(f'Max depth vs 5 fold cv score')
# plt.show()

#### Looks like best max_depth is around 15
* Review what other are good parameters to vary

In [ ]:
model_dt.feature_importances_
indices = np.argsort(model_dt.feature_importances_)[::-1]
g = sns.barplot(y=train.columns[indices],x = model_dt.feature_importances_[indices] , orient='h')
plt.title('Feature Importances for Decision Tree');

## Random Forest

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(n_estimators = 400, random_state=config.seed)
preds_rf,scores_rf,oof_score_rf,oof_rf = run_cv_and_predict(train,test,features,model_rf,config.seed,verbose=False)

# Default rf, 100 est: MAE score: 0.65300, Stdev: 0.01621, OOF score: 0.65100
# 50 est: MAE score: 0.65800, Stdev: 0.01789, OOF score: 0.66000
# 200 est: MAE score: 0.65150, Stdev: 0.01323, OOF score: 0.65230
# 400 est: MAE score: 0.64783, Stdev: 0.01812, OOF score: 0.64700, lb = 

In [ ]:
model_rf.get_params()

## Extra Trees

In [ ]:
##time
from sklearn.ensemble import ExtraTreesRegressor

model_etc = ExtraTreesRegressor(criterion='absolute_error',
                                n_estimators=200,
                                max_depth=12,
                                max_features=None,
                                n_jobs = 4,
                                random_state=config.seed)

preds_etc,scores_etc,oof_score_etc,oof_etc = run_cv_and_predict(train,test,features,model_etc,config.seed,verbose=True)

# default 100 est, MAE score: 0.65010, Stdev: 0.02108, OOF score: 0.65100, lb = 
# 200 est: MAE score: 0.65085, Stdev: 0.01528, OOF score: 0.65000, 
# default with criterion of absolute error. .66640 (worse)
# use new params from spreadsheet, 200 est, 12 md, max_features = None, cv = .54529


In [ ]:
model_etc.get_params()

# XGBoost

In [ ]:
%%time
from xgboost import XGBRegressor

# results = []
# for est in range(20,60):

model_xgb = XGBRegressor(n_estimators=46, random_state=config.seed)
preds_xgb,scores_xgb,oof_score_xgb,oof_xgb = run_cv_and_predict(train,test,features,model_xgb,config.seed,verbose=False)
#     results.append([est,np.mean(scores_xgb)])


# results = pd.DataFrame(results, columns=['estimators','score'])
# results.head()

# default: .67487
# 46 estimators: .65363

In [ ]:
#plt.plot(results.estimators,results.score)

First run looks like 46 estimator is min

### Takeways
* Seems to like lower estimator values
* do a 3 way grid opimization next

## Catboost

In [ ]:
%%time
from catboost import CatBoostRegressor

results = []
for est in [100,200,300,400,500,750,1000]:
    for md in [3,4,5,6,7,8,10,12,14]:
        for lr in [.01,.02,.03,.04,.05,.06,.08,.1]:
            model_cat = CatBoostRegressor(random_state=config.seed, 
                                          verbose=0, 
                                          eval_metric='MAE',
                                          n_estimators=est,
                                          max_depth=md,
                                          learning_rate=lr
                                          )  # maybe quantile for eval metric
            
            print(f'est: {est},max depth: {md}, learning rate: {lr}')
            preds_cat,scores_cat,oof_score_cat,oof_cat = run_cv_and_predict(train,test,features,model_cat,config.seed,verbose=False)
            results.append([est,md,lr,np.mean(scores_cat)])

# default :.65412

In [ ]:
results_df = pd.DataFrame(results,columns=['estimators','max_depth','learning_rate','cv_score'])
results_df.head()

In [ ]:
model_cat.get_all_params()

## Prediction Selection

In [ ]:
final_preds = np.mean(preds_xgb,axis=1)

# for svm, goes negative, clamp to zero
final_preds[final_preds < 0.0] = 0.0
final_preds[final_preds > 10.0] = 10.0

# Submission

In [ ]:
ss.Hardness = final_preds
ss.to_csv('submission.csv',index=False)
ss.head()

In [ ]:
plt.hist(ss.Hardness, bins=30);